# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [4]:
# frequency table for prestige and whether or not someone was admitted
print len(df)

397


In [5]:
instance,count= np.unique(df.prestige, return_counts=True)
print np.asarray((instance,count)).T
# this is one way of doing it

[[   1.   61.]
 [   2.  148.]
 [   3.  121.]
 [   4.   67.]]


In [6]:
from scipy.stats import itemfreq
itemfreq(df.prestige)
# this is another way of doing this

array([[   1.,   61.],
       [   2.,  148.],
       [   3.,  121.],
       [   4.,   67.]])

In [7]:
table=pd.DataFrame(count,index=instance, columns=['# of instances of prestige variable'])
print table
# this creates an actual table, nicely labeled, as opposed to just an array

     # of instances of prestige variable
1.0                                   61
2.0                                  148
3.0                                  121
4.0                                   67


In [8]:
instance2,count2=np.unique(df.admit, return_counts=True)
table2=pd.DataFrame(count2,index=['Not admitted (0)','Admitted (1)'], columns=['# of applicants in each category'])
print table2

                  # of applicants in each category
Not admitted (0)                               271
Admitted (1)                                   126


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [9]:
prestige_dummies=pd.get_dummies(df.prestige)
prestige_dummies.columns=['Prestige_1',"Prestige_2","Prestige_3",'Prestige_4']
print prestige_dummies
# Have created the dummy variables and put them in their own separate dataframe named prestige_dummies- will be able to join this to any other df's later...



     Prestige_1  Prestige_2  Prestige_3  Prestige_4
0           0.0         0.0         1.0         0.0
1           0.0         0.0         1.0         0.0
2           1.0         0.0         0.0         0.0
3           0.0         0.0         0.0         1.0
4           0.0         0.0         0.0         1.0
5           0.0         1.0         0.0         0.0
6           1.0         0.0         0.0         0.0
7           0.0         1.0         0.0         0.0
8           0.0         0.0         1.0         0.0
9           0.0         1.0         0.0         0.0
10          0.0         0.0         0.0         1.0
11          1.0         0.0         0.0         0.0
12          1.0         0.0         0.0         0.0
13          0.0         1.0         0.0         0.0
14          1.0         0.0         0.0         0.0
15          0.0         0.0         1.0         0.0
16          0.0         0.0         0.0         1.0
17          0.0         0.0         1.0         0.0
18          

#### 2.2 When modeling our class variables, how many do we need? 



Answer: Since there are 4 outcomes, we need to use 3 of them in any analysis we conduct, since whenever a categorical variable with n outcomes is encoded into multiple 1/0 dummy variables, n-1 of them need to be included in analysis (since by definition knowing the value of 3 of the variables also makes the 4th known, and therefore incorporating all 4 into analysis would be redundant and skew any analysis 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [10]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(prestige_dummies['Prestige_1'])
print handCalc.head()

   admit    gre   gpa  Prestige_1
0      0  380.0  3.61         0.0
1      1  660.0  3.67         0.0
2      1  800.0  4.00         1.0
3      1  640.0  3.19         0.0
4      0  520.0  2.93         0.0


In [11]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
crosstab=pd.crosstab(handCalc.admit,handCalc.Prestige_1)
print crosstab

Prestige_1  0.0  1.0
admit               
0           243   28
1            93   33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [15]:
# the odds would be 33:28, since it's probability of admittance divided by probability of non-admittance
odds1=float(crosstab[1][1])/float(crosstab[1][0])
print odds1

1.17857142857


The odds would be equal to ~ 1.18

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [16]:
#the odds would be 93:243
odds2= float(crosstab[0][1])/float(crosstab[0][0])
print odds2

0.382716049383


#### 3.3 Calculate the odds ratio

In [17]:
# the odds ratio would be the division of the odds of being admitted if attending #1 divided by the odds of being admitted if NOT attending a #1
print odds1/odds2

3.07949308756


The odds ratio is ~3.08

#### 3.4 Write this finding in a sentence: 

Answer: The odds of being admitted for a student that attended a top tier (Prestige=1) school are ~3.08 larger than the odds for a student that did not attend a top tier (Prestige=1) school


#### 3.5 Print the cross tab for prestige_4

In [18]:
handCalc=handCalc.join(prestige_dummies['Prestige_4'])
crosstab2=pd.crosstab(handCalc.admit,handCalc.Prestige_4)
print crosstab2

Prestige_4  0.0  1.0
admit               
0           216   55
1           114   12


#### 3.6 Calculate the OR 

In [19]:
odds_1=float(crosstab2[1][1])/float(crosstab2[1][0])
odds_2=float(crosstab2[0][1])/float(crosstab2[0][0])
print odds_1/odds_2

0.413397129187


#### 3.7 Write this finding in a sentence

Answer: The odds of being admitted for a student that attended the least prestigious category of school (Prestige=4) are ~.41 (e.g. significantly lower) than the odds for a student that attended any schools that were more prestigious (Prestige 1-3) 

## Part 4. Analysis

In [20]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(prestige_dummies['Prestige_1'])
data=data.join(prestige_dummies.Prestige_2)
data=data.join(prestige_dummies.Prestige_3)

print data.head()

   admit    gre   gpa  Prestige_1  Prestige_2  Prestige_3
0      0  380.0  3.61         0.0         0.0         1.0
1      1  660.0  3.67         0.0         0.0         1.0
2      1  800.0  4.00         1.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         0.0
4      0  520.0  2.93         0.0         0.0         0.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [21]:
# manually add the intercept
data['intercept'] = 1.0
print data.head()

   admit    gre   gpa  Prestige_1  Prestige_2  Prestige_3  intercept
0      0  380.0  3.61         0.0         0.0         1.0        1.0
1      1  660.0  3.67         0.0         0.0         1.0        1.0
2      1  800.0  4.00         1.0         0.0         0.0        1.0
3      1  640.0  3.19         0.0         0.0         0.0        1.0
4      0  520.0  2.93         0.0         0.0         0.0        1.0


#### 4.1 Set the covariates to a variable called train_cols

In [22]:
train_cols=data[['gre','gpa','Prestige_1','Prestige_2','Prestige_3','intercept']]
admit=data.admit

#### 4.2 Fit the model

In [23]:
model=sm.Logit(data.admit,train_cols)

In [24]:
model_fit=model.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


In [25]:
# above was the sm version- want to double check results against the sklearn version
from sklearn import tree, cross_validation, linear_model, metrics

In [26]:
test_model=linear_model.LogisticRegression(fit_intercept=False)
fittedmodel=test_model.fit(train_cols,admit)
print fittedmodel.intercept_
print fittedmodel.coef_

0.0
[[  1.58889206e-03   1.84630696e-04   1.16761197e+00   5.26947989e-01
   -3.80822681e-02  -2.07018745e+00]]


In [27]:
# open question why this would give different results- presume it has something to do w/ the way the intercept is handled?

#### 4.3 Print the summary results

In [83]:
print model_fit.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sat, 23 Jul 2016   Pseudo R-squ.:                 0.08166
Time:                        20:31:00   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
Prestige_1     1.5534      0.417      3.721      0.0

In [84]:
oddsratios=model_fit.params
oddsratios=pd.DataFrame(oddsratios)
oddsratios.rename(columns={0:'Coefficient'},inplace=True)
print oddsratios

            Coefficient
gre            0.002218
gpa            0.779337
Prestige_1     1.553411
Prestige_2     0.873274
Prestige_3     0.214733
intercept     -5.430265


In [85]:
confint=pd.DataFrame(model_fit.conf_int())
confint.rename(columns={0:"CI- Lower",1:'CI- Upper'},inplace=True)
print confint

            CI- Lower  CI- Upper
gre          0.000074   0.004362
gpa          0.127619   1.431056
Prestige_1   0.735197   2.371624
Prestige_2   0.153432   1.593115
Prestige_3  -0.554669   0.984135
intercept   -7.664377  -3.196152


In [86]:
oddsratios=oddsratios.join(confint)

In [87]:
oddsratios['OR']=''
print oddsratios

            Coefficient  CI- Lower  CI- Upper OR
gre            0.002218   0.000074   0.004362   
gpa            0.779337   0.127619   1.431056   
Prestige_1     1.553411   0.735197   2.371624   
Prestige_2     0.873274   0.153432   1.593115   
Prestige_3     0.214733  -0.554669   0.984135   
intercept     -5.430265  -7.664377  -3.196152   


In [88]:
def calc_OR(row):
    return np.exp(row.Coefficient)

In [91]:
oddsratios['OR']=oddsratios.apply(calc_OR,axis=1)


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [92]:
print oddsratios

            Coefficient  CI- Lower  CI- Upper        OR
gre            0.002218   0.000074   0.004362  1.002221
gpa            0.779337   0.127619   1.431056  2.180027
Prestige_1     1.553411   0.735197   2.371624  4.727566
Prestige_2     0.873274   0.153432   1.593115  2.394738
Prestige_3     0.214733  -0.554669   0.984135  1.239531
intercept     -5.430265  -7.664377  -3.196152  0.004382


#### 4.5 Interpret the OR of Prestige_2

Answer: The likelihood of admittance for someone that attended a prestige 2 school (2nd tier) is ~2.39 times greater than the likelihood of admittance for someone that did not attend a prestige 2 school

#### 4.6 Interpret the OR of GPA

Answer: An increase in GPA by one point increases the likelihood of admittance by a factor of 2.18, relative to the likelihood of admittance without an increase in GPA by one point

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [99]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [104]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [127]:
print combos

         0         1    2    3
0    220.0  2.260000  1.0  1.0
1    220.0  2.260000  2.0  1.0
2    220.0  2.260000  3.0  1.0
3    220.0  2.260000  4.0  1.0
4    220.0  2.453333  1.0  1.0
5    220.0  2.453333  2.0  1.0
6    220.0  2.453333  3.0  1.0
7    220.0  2.453333  4.0  1.0
8    220.0  2.646667  1.0  1.0
9    220.0  2.646667  2.0  1.0
10   220.0  2.646667  3.0  1.0
11   220.0  2.646667  4.0  1.0
12   220.0  2.840000  1.0  1.0
13   220.0  2.840000  2.0  1.0
14   220.0  2.840000  3.0  1.0
15   220.0  2.840000  4.0  1.0
16   220.0  3.033333  1.0  1.0
17   220.0  3.033333  2.0  1.0
18   220.0  3.033333  3.0  1.0
19   220.0  3.033333  4.0  1.0
20   220.0  3.226667  1.0  1.0
21   220.0  3.226667  2.0  1.0
22   220.0  3.226667  3.0  1.0
23   220.0  3.226667  4.0  1.0
24   220.0  3.420000  1.0  1.0
25   220.0  3.420000  2.0  1.0
26   220.0  3.420000  3.0  1.0
27   220.0  3.420000  4.0  1.0
28   220.0  3.613333  1.0  1.0
29   220.0  3.613333  2.0  1.0
..     ...       ...  ...  ...
370  800

#### 5.1 Recreate the dummy variables

In [103]:

# recreate the dummy variables

# keep only what we need for making predictions


In [131]:
prestige1=np.linspace(data['Prestige_1'].min(),data['Prestige_1'].max(),10)
prestige2=np.linspace(data['Prestige_2'].min(),data['Prestige_2'].max(),10)
prestige3=np.linspace(data['Prestige_3'].min(),data['Prestige_3'].max(),10)
intercept2=np.linspace(data['intercept'].min(),data['intercept'].max(),10)
#are these treated the same way even if binary? need to find out...


In [144]:
combos2 = pd.DataFrame(cartesian([gres, gpas,prestige1,prestige2,prestige3,intercept2]))
print combos2

            0     1    2    3         4    5
0       220.0  2.26  0.0  0.0  0.000000  1.0
1       220.0  2.26  0.0  0.0  0.000000  1.0
2       220.0  2.26  0.0  0.0  0.000000  1.0
3       220.0  2.26  0.0  0.0  0.000000  1.0
4       220.0  2.26  0.0  0.0  0.000000  1.0
5       220.0  2.26  0.0  0.0  0.000000  1.0
6       220.0  2.26  0.0  0.0  0.000000  1.0
7       220.0  2.26  0.0  0.0  0.000000  1.0
8       220.0  2.26  0.0  0.0  0.000000  1.0
9       220.0  2.26  0.0  0.0  0.000000  1.0
10      220.0  2.26  0.0  0.0  0.111111  1.0
11      220.0  2.26  0.0  0.0  0.111111  1.0
12      220.0  2.26  0.0  0.0  0.111111  1.0
13      220.0  2.26  0.0  0.0  0.111111  1.0
14      220.0  2.26  0.0  0.0  0.111111  1.0
15      220.0  2.26  0.0  0.0  0.111111  1.0
16      220.0  2.26  0.0  0.0  0.111111  1.0
17      220.0  2.26  0.0  0.0  0.111111  1.0
18      220.0  2.26  0.0  0.0  0.111111  1.0
19      220.0  2.26  0.0  0.0  0.111111  1.0
20      220.0  2.26  0.0  0.0  0.222222  1.0
21      22

In [145]:
# why are there so many rows? does each additional column drastically expand this (e.g. basically exponential growth?)
# also, why does column 4 have actual values between 0 and 1 while the others don't 

#### 5.2 Make predictions on the enumerated dataset

In [143]:
output=model_fit.predict(combos2)
print output

[ 0.03989032  0.03989032  0.03989032 ...,  0.89121621  0.89121621
  0.89121621]


#### 5.3 Interpret findings for the last 4 observations

Answer: The last 4 observations all seem to indicate an admit value of .89, implying that with those inputs (gre of 800, GPA of 4, and then a score of 1 for all the binaries the likelihood of admit would be .89. However, this result appears skewed, since all of the dummy variables cannot simultaneously be 1. 

Note: Surely there is a better way of doing this/dropping the "impossible" values, but I'm not quite following the logic behind the Cartesian distribution so I tried my best above, realizing that the output is somewhat flawed. Looking forward to reviewing this last section in more detail!


## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.